#1. Salary Prediction Project - Data Collection Part 1
by: Cindy Suyitno

The data will be collected from:

https://glints.com/id/opportunities/jobs/explore?country=ID&locationName=Indonesia&gclid=Cj0KCQjwkruVBhCHARIsACVIiOwkqvkaoBKtrxO2n5JAlucbZD1OCfdhStJ7iMXIChg6ou--n6m-xAkaAj8DEALw_wcB&utm_source=google&utm_medium=cpc&utm_campaign=ID|Marketplace|Search|Brand&gclid=Cj0KCQjwkruVBhCHARIsACVIiOwkqvkaoBKtrxO2n5JAlucbZD1OCfdhStJ7iMXIChg6ou--n6m-xAkaAj8DEALw_wcB 

aka Glints webpage, one of the famous website with Job vacancies for Indonesian.

File that is extracted from this part:
- DataCollection_Glints.csv (--> will be used in part 3. Data Cleaning)

In [ ]:
#There are some modules need to be install for this data collection process
!pip install beautifulsoup4
!pip install lxml
!pip install selenium
!pip install CurrencyConverter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 4.4 MB/s 
     |████████████████████████████████| 358 kB 52.2 MB/s 
     |████████████████████████████████| 138 kB 57.4 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 4.0 MB 45.1 MB/s 
     |████████████████████████████████| 58 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires u

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 547 kB 4.4 MB/s 


In [ ]:
#since this file is used in googlecollab, there are some prior setting as well for the chromedriver, which is esential in this step
!apt-get update
!apt install chromium-chromedriver

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRele

In [ ]:
#this syntax will mount my google colab file to my google drive
from google.colab import drive
drive.mount ('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks 
#change directory

/content/drive/MyDrive/Colab Notebooks


In [ ]:
#library imports
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import os
from random import randint
from time import sleep
from currency_converter import CurrencyConverter

In [ ]:
#these bunch of codes are important to establish webdriver.Chrome, which is a python tool for web scraping
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#this is the data href i scrapped on 6/9/2022, on glints.com (which has 300 pages)
glints_site = []
for a in range(300):
  glints_site.append('https://glints.com/id/opportunities/jobs/explore?country=ID&locationName=Indonesia&gclid=CjwKCAjwkYGVBhArEiwA4sZLuO7UuluK-tMrs4ZCZp-E7ZMeembyo3lSz9hwCQofjoeLs0MBqVmsNBoC394QAvD_BwE&gclid=CjwKCAjwkYGVBhArEiwA4sZLuO7UuluK-tMrs4ZCZp-E7ZMeembyo3lSz9hwCQofjoeLs0MBqVmsNBoC394QAvD_BwE&utm_source=google&utm_medium=cpc&utm_campaign=ID%7CMarketplace%7CSearch%7CBrand&page='+str(a+1))

In [ ]:
#there are 4 items i want to scrap from this website: job name, city, salary, and years of experiences
#these are the class name of those items (can be retrieved from click right -> Inspect to the text in the website)
#job name = CompactOpportunityCardsc__JobTitle-sc-1y4v110-7 bVuGlP
#kota, gaji, yrs of experience = CompactOpportunityCardsc__OpportunityInfo-sc-1y4v110-13 ikxvyY

my_table_data1_1 = []
my_table_data1_2 = []
for i in range(0,300):
  url = glints_site[i]
  wd.get(url)
  sleep(randint(1,3)) #slow down process to avoid error
  soup = BeautifulSoup(wd.page_source, 'html.parser')
  my_table = soup.find_all(class_=['CompactOpportunityCardsc__JobTitle-sc-1y4v110-7 bVuGlP'])
  for data in my_table:
      my_table_data1_1.append(data.get_text())
  my_table2 = soup.find_all(class_=['CompactOpportunityCardsc__OpportunityInfo-sc-1y4v110-13 ikxvyY'])
  for data in my_table2:
      my_table_data1_2.append(data.get_text()) 

In [ ]:
#assigning each item to salary, city, and years
cc = CurrencyConverter()
currency = list(cc.currencies)

def is_this_salary(a):
  for y in range(len(currency)):
    if currency[y] in str(a):
      return True
    elif 'salary' in str(a):
      return True
  else:
    return False

salary = []
city = []
years = []
x=0

try:
  for x in range(len(my_table_data1_2)): 
    if ('tahun' not in my_table_data1_2[x])&(not(is_this_salary(my_table_data1_2[x]))):
      city.append(my_table_data1_2[x])
      if (is_this_salary(my_table_data1_2[x+1])): 
        pass
      elif 'tahun' in my_table_data1_2[x+1]:
        salary.append('')
      elif bool(my_table_data1_2[x+1])==False:
        pass
      else:
        salary.append('')
        years.append('')
    elif (is_this_salary(my_table_data1_2[x])):
      salary.append(my_table_data1_2[x])
      if 'tahun' in my_table_data1_2[x+1]:
        pass
      elif ('tahun' not in my_table_data1_2[x+1])&(not(is_this_salary(my_table_data1_2[x+1]))):
        years.append('')
      elif bool(my_table_data1_2[x+1])==False:
        pass
      else:
        city.append('')
        years.append('')
    elif 'tahun' in my_table_data1_2[x]:
      years.append(my_table_data1_2[x])
      if ('tahun' not in my_table_data1_2[x+1])&(not(is_this_salary(my_table_data1_2[x+1]))):
        pass
      elif (is_this_salary(my_table_data1_2[x+1])):
        city.append('')
      elif bool(my_table_data1_2[x+1])==False:
        pass
      else:
        city.append('')
        salary.append('')
except IndexError: pass #avoid index error

In [ ]:
#taking 8000 data
df = pd.DataFrame()
df['Job_Title'] = my_table_data1_1[0:8000]
df['City'] = city[0:8000]
df['Yrs_of_Exp'] = years[0:8000]
df['Salary'] = salary[0:8000]
df

,Job_Title,City,Yrs_of_Exp,Salary
0,Customer Service,"Senen, Jakarta",1 – 3 tahun,IDR 4.500.000 - 5.000.000
1,Project Admin Legal Staff,"Tanah Abang, Jakarta",1 – 3 tahun,IDR 4.500.000 - 6.000.000
2,HR - Admin,Jakarta,Kurang dari setahun,IDR 4.700.000 - 5.000.000
3,Admin Gudang,Jakarta,1 – 3 tahun,
4,Staff Administrasi,Bandung,Kurang dari setahun,IDR 3.500.000 - 4.000.000
...,...,...,...,...
7995,Nursery/Kindergarten Teacher Assistant,"Kramat Jati, Jakarta",1 – 3 tahun,IDR 4.500.000 - 6.500.000
7996,Accounting&Finance Staff,"Cengkareng, Jakarta",1 – 3 tahun,IDR 4.641.854 - 6.000.000
7997,TikTok & Instagram Talent,Bekasi,1 – 3 tahun,IDR 5.000.000 - 6.000.000
7998,Account Manager,Jakarta,1 – 3 tahun,IDR 3.500.000 - 4.000.000


In [ ]:
#exporting to csv file
df.to_csv('DataCollection_Glints.csv',index=False)